# Introduction to Partial Differential Equations
---

## Chapter 2: Elliptic PDEs, Poisson’s Equation, and a Two-Point Boundary Value Problem 
---

## Want to use Colab? [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp2/Chp2Sec2.ipynb)

---

## Prepping the environment for interactive plots in Colab
---

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab - installing missing packages')
    !pip install ipympl
    from IPython.display import clear_output
    clear_output()
    exit()
else:
    print('Not running on CoLab - assuming environment has necessary packages')

In [ ]:
%matplotlib widget
if 'google.colab' in str(get_ipython()):
    from google.colab import output
    output.enable_custom_widget_manager()

## Creative Commons License Information
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/80x15.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Introduction to Partial Differential Equations: Theory and Computations</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" property="cc:attributionName" rel="cc:attributionURL">Troy Butler</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" rel="dct:source">https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations</a>.

## Section 2.2:  A Finite Difference Approximation in 1-D
---

In this section, we will use Taylor series to derive a centered finite difference scheme leading to a finite difference approximation of the 2-pt BVP  $-u''(x)=f(x)$ for $x\in(0,1)$ with $u(0)=u(1)=0$. Specifically, we will derive an $A\in\mathbb{R}^{n\times n}$ of the form

$$
\large A = \begin{pmatrix}
                    2 & -1 & 0 & \cdots & 0 \\
                    -1 & 2 & -1 & \ddots & \vdots \\
                    0 & \ddots & \ddots & \ddots & 0 \\
                    \vdots & \ddots & -1 & 2 & -1 \\
                    0 & \cdots & 0 & -1 & 2
           \end{pmatrix}
$$

and a data vector $b$ of the form

$$
\large b = h^2\begin{pmatrix}
                        f(x_1) \\
                        f(x_2) \\
                        \vdots \\
                        f(x_n)
               \end{pmatrix}
$$


that gives us the discrete problem $Av=b$ where $v\in\mathbb{R}^n$ has the property that $v_j\approx u(x_j)$.

- The $h^2$ comes from discretizing the differential operator $L=\frac{d^2}{dx^2}$ with the centered finite difference scheme. 

  - It is perhaps slightly strange that we put this on the data vector instead of on $A$.

  - In [Section 2.3](Chp2Sec3.ipynb), we see that $\frac{1}{h^2}A$ defines the difference operator $L_h$ that is the discrete counterpart to $L$.
  
  - So, why don't we "attach" this $h^2$ on $A$ in the form of a multiplicative factor of $1/h^2$? 
  
    - Well, remember that we need to *solve* $Av=b$ for $v$. So, even if we multiplied $A$ by $1/h^2$, we then need to "undo" this when solving for $v$. This saves us computational effort.


- How do we *solve* $Av=b$ in practice? Gaussian elimination is a method that should be familiar from undergraduate linear algebra.

  - If not, it is straightforward if not a bit messy (you may want to review this here https://en.wikipedia.org/wiki/Gaussian_elimination). 
  
  - The idea is to reduce the system of linear equations into row-echelon form (https://en.wikipedia.org/wiki/Row_echelon_form) and use back-substitution to then determine the vector $v$ such that $Av=b$.
  

**Key question:** Since formally we can write $v=A^{-1}b$ (assuming $A^{-1}$ exists), why we do not simply just construct the inverse of the matrix?

**The answer:** Because that is ***expensive*** in terms of [FLOPS](https://en.wikipedia.org/wiki/FLOPS) and thanks to Gaussian elimination, completely unnecessary in determining $v$ with less FLOPS.

Think of FLOPS as a computational currency that you must pay in order to solve a problem. If we can solve the same problem with less FLOPS in method A vs. method B, then we say method A is *computationally cheaper* than method B.

**The algorithm:** While you could code up your own version of Gaussian elimination, there are many available computational libraries that will do this for you efficiently. 
We show how using [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html) below on an example.

---
### <a id='Section2.2.1'>Section 2.2.1: A centered finite difference approximation to second-order derivatives</a>
---

Students may find reviewing [Section 1.2.3 in the Section 1.2 notebook](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp1/Chp1Sec2.ipynb) useful before proceeding. 

Here, assume that for a given $x\in\mathbb{R}$ that $g\in\mathcal{C}^4([x-h_\max, x+h_\max])$, then for any $0<h\leq h_\max$, Taylor's theorem gives

$$
    g(x+h) = g(x) + g'(x)h + \frac{h^2}{2}g''(x) + \frac{h^3}{6}g^{(3)}(x) + \frac{h^4}{24}g^{(4)}(\xi_1), 
$$

where $\xi_1\in[x,x+h]$. Similarly, for the same $h>0$, we have

$$
    g(x-h) = g(x) - g'(x)h + \frac{h^2}{2}g''(x) - \frac{h^3}{6}g^{(3)}(x) + \frac{h^4}{24}g^{(4)}(\xi_2), 
$$

where $\xi_2\in[x-h, x]$. If we *add* these two expressions together, the terms associated with $h$ and $h^3$ cancel out due to the opposing signs, and we get

$$
    g(x+h) + g(x-h) = 2g(x) + h^2 g''(x) + \frac{h^4}{24}\left[g^{(4)}(\xi_1) + g^{(4)}(\xi_2) \right].
$$

Now, we solve for $g''(x)$ but subtracting $2g(x)$ and dividing through by $h^2$ to get

$$
    g''(x) + E_h(x) = \frac{g(x+h)-2g(x) + g(x-h)}{h^2}, 
$$

where

$$
    E_h(x) := \frac{h^2}{24}\left[g^{(4)}(\xi_1) + g^{(4)}(\xi_2) \right].
$$

**Remarks:**

- $E_h(x)$ is at least $\mathcal{O}(h^2)$ because we assumed $g\in\mathcal{C}^4(\mathbb{R})$, which means that for any finite $h>0$, $|g^{(4)}|$ is a continuous function on $[x-h, x+h]$, so it must obtain a maximum value, i.e., 
<br><br>
$$
    |E_h(x)| \leq \frac{M_g h^2}{12}, 
$$
<br>
where 
<br><br>
$$
    M_g := \sup_{\xi \in [x-h_\max,x+h_\max]} | g^{(4)} |
$$
<br><br>
and we can actually replace the $\sup$ with $\max$.


- Given this information about $E_h(x)$, the following approximation is "reasonable" for any $g$ that is four-times continuously differentiable on an interval as long as $h$ is sufficiently small:

$$
    g''(x) \approx \frac{g(x+h)-2g(x) + g(x-h)}{h^2}.
$$


- If $g$ is a function such that $g^{(4)}\equiv 0$, then $E_h(x)=0$ and the above approximation becomes an equality. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def pp_approx(g, x, h=0.1):
    g_pp = (g(x+h) - 2*g(x) + g(x-h)) / h**2
    return g_pp

Let $g(x) = e^{-x^2}\sin(\pi x)$, which produces a somewhat interesting plot. Let's play with this below to study the difference approximation and the practical limitations on making $h$ too small.

In [ ]:
import sympy as sp

In [ ]:
x = sp.symbols('x')

In [ ]:
g = sp.exp(-x**2)*sp.sin(sp.pi * x)

In [ ]:
g

In [ ]:
g_pp = g.diff(x, 2)

In [ ]:
g_pp

In [ ]:
from sympy.utilities.lambdify import lambdify

In [ ]:
g_eval = lambdify(x, g)

g_pp_eval = lambdify(x, g_pp)

In [ ]:
def plot_gpp(g, g_pp, n, h=0.1):
    
    plt.figure(0)
    plt.clf()
    
    x = np.linspace(-3, 3, n)
    
    plt.plot(x, g_pp(x), ls=':', c='b', label="exact $g''$")
    plt.plot(x, pp_approx(g, x, h), ls='-.', c='r', label=r"$\approx$ $g''$")
    plt.legend()
    plt.show()

In [ ]:
%matplotlib widget

%reset -f out 

# Check out what happens when $h<1e-7$

interact_manual(plot_gpp, 
            g = fixed(g_eval),
            g_pp = fixed(g_pp_eval),
            h=widgets.FloatLogSlider(value=0.01, min=-9, max=-1, base=10, step=1),
            n=widgets.IntSlider(value=100, min=50, max=1000, step=50))

In [ ]:
g_pppp = g.diff(x, 4)

In [ ]:
g_pppp_eval = lambdify(x, g_pppp)

In [ ]:
x = np.linspace(-3, 3, 1000)
g_pppp_max = np.abs(np.max(g_pppp_eval(x)))
print(g_pppp_max)

In [ ]:
def plot_gpp_error_and_bounds(g, g_pp, g_pppp_max, n, h=0.1):
    
    plt.figure(1)
    plt.clf()
    
    x = np.linspace(-3, 3, n)
    
    plt.plot(x, np.abs( g_pp(x) - pp_approx(g, x, h) ), ls=':', c='b', label="Error")
    
    plt.plot([-3, 3], np.array([g_pppp_max, g_pppp_max])*h**2/12, ls='-.', c='r', label=r"$M_gh^2/12$")
    plt.legend()
    plt.show()

In [ ]:
%matplotlib widget

%reset -f out 

# Check out what happens when $h<1e-4$

interact_manual(plot_gpp_error_and_bounds, 
            g = fixed(g_eval),
            g_pp = fixed(g_pp_eval),
            g_pppp_max = fixed(g_pppp_max),
            h=widgets.FloatLogSlider(value=0.01, min=-9, max=-1, base=10, step=1),
            n=widgets.IntSlider(value=100, min=50, max=1000, step=50))

<mark>Students are encouraged to use the approach shown in [Section 1.2](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp1/Chp1Sec2.ipynb) for estimating rates of convergence for $0.1>h>1e-7$ (and also $0.1>h>1e-4$). </mark>

---
### <a id='Section2.2.2'>Section 2.2.2: The finite difference approximation of $-u''(x)=f(x)$ for $x\in(0,1)$ with $u(0)=u(1)=0$</a>
---

The steps/ideas for deriving the "prototypical" finite difference approximation we seek are as follows.

1. Partition $[0,1]$ into $n+1$ subintervals of equal length $h:=1/(n+1)$. 

   This gives $n+2$ equally spaced points, denoted by $\{x_j\}_{j=0}^{n+1}$, where $x_j=jh$.
   
   **Why use $n+1$ subintervals instead of $n$ subintervals?** Remember, we know $u(x_0)=u(0)=0$ and $u(x_{n+1})=u(1)=1$. Thus, this is just another way of saying that we want to consider $n$ interior points $x_1,\ldots, x_n$ corresponding to the unknown values of $u(x_j)$ that we seek to approximate. If we used $n$ subintervals, then we would have $n-1$ unknown values. Thus, using $n+1$ subintervals is just an elegant way of not having to write $n-1$ to describe the number of unknowns.
   
   Let $\{v_j\}_{j=0}^{n+1}$ define the approximation of $u$ at the points $\{x_j\}_{j=0}^{n+1}$. 
   
2. How should we determine $v\in\mathbb{R}^{n+1}$ so that $v_j \approx u(x_j)$?

   By requiring that $v_0=0=v_{n+1}$, there is no approximation error at the endpoints. Fantastic. 
   
   While we don't know what $u$ is at $x_1,\ldots, x_n$, we do know that $-u''=f$ at all of these points. So, we use the centered finite difference approximation derived above in [Section 2.2.1](#Section2.2.1) and require that
   
   $$
       - \frac{v_{j-1} - 2v_j + v_{j+1}}{h^2} = f(x_j), \ \text{ for } \ j=1,\ldots, n.
   $$

   Note that this gives $n$ equations for $n$ unknowns and since we required $v_0=0=v_n$, then for the $j=1$ and $j=n$ equations, we actually have
   
   $$
       - \frac{-2v_1 + v_2}{h^2} = f(x_1), \ \text{ and } \ - \frac{v_{n-1} - 2v_n }{h^2} = f(x_n).
   $$
   
   <mark>An important note is that the above two questions are clearly the ones to change if we have different boundary conditions or values applied to the problem.</mark>

3. We can organize the system of $n$ equations for $n$ unknowns as the matrix-vector equation
<br><br>
$$
\large Av=b, \ \text{ where } \ A = \begin{pmatrix}
                    2 & -1 & 0 & \cdots & 0 \\
                    -1 & 2 & -1 & \ddots & \vdots \\
                    0 & \ddots & \ddots & \ddots & 0 \\
                    \vdots & \ddots & -1 & 2 & -1 \\
                    0 & \cdots & 0 & -1 & 2
                \end{pmatrix},
            \
            \
             b = h^2\begin{pmatrix} 
                        f(x_1) \\
                        f(x_2) \\
                        \vdots \\
                        f(x_n)
                    \end{pmatrix}
$$
<br><br>
and $v\in\mathbb{R}^n$ represents $v=(v_1, v_2, \ldots, v_n)^\top\approx (u(x_1), u(x_2), \ldots, u(x_n))^\top$.

4. We analyze the accuracy and convergence rate of this method using the method of manufactured solutions and the error function 
<br><br>
$$
    \large E_h := \max_{1\leq j\leq n} |u(x_j) - v_j|.
$$
<br><br>
Note that if we change one of the boundary condition types (e.g., to Neumann or Robin), then we would adjust the system of equations by adding either one or two more equations that we need to solve to determine $v_0$ or $v_{n+1}$ (possibly both), and we should similarly adjust the definition of $E_h$ to include the approximation error we make at these endpoints.

---
### Section 2.2.3: General code implementation and the method of manufactured solutions
---

- We create a function `make_A` for making the matrix $A$. 

  We could use sparse matrix packages for this since most of the components of $A$ are zero, but we eschew that for now to keep things as "familiar" as possible (at least initially) by using standard `numpy` arrays. This is less memory efficient, but we are not going to deal with $n$ values that make this problematic.

- We do *not* create a function for making $b$ because we assume we can evaluate $f$ on an array to get an array. In other words, the function $f$ serves to make $b$ directly.

- As discussed previously in other notebooks, the method of manufactured solutions refers to the approach where we verify and study code by starting with a solution to the problem (i.e., start with some $u$ that we "manufacture" essentially out of thin air), derive the problem conditions that would lead to such a manufactured solution (i.e., plug the solution into the problem to determine what the problem needs to look like to give that solution), and then we *know* how to compute the error to check the code/approach because we actually have the *exact* solution.

We show all of this below.

In [ ]:
def make_A(n):
    A = np.zeros((n,n))
    np.fill_diagonal(A,2)
    A += np.diag(-np.ones(n-1),k=1)
    A += np.diag(-np.ones(n-1),k=-1)
    return A

In [ ]:
n = 5  # just for illustrative purposes
A = make_A(n)
print(A)

In [ ]:
# Suppose we make $u$ the $g$ we saw in Section 2.1.1 above
x = sp.symbols('x')
u = sp.exp(-x**2)*sp.sin(sp.pi * x)
u

The $\sin(\pi x)$ is zero at both $x=0$ and $x=1$, so $u(0)=u(1)=0$, which we can also verify below.

In [ ]:
u.subs({x:0})

In [ ]:
u.subs({x:1})

We need $-u''=f$, so let's figure out what $f$ is.

In [ ]:
f = -u.diff(x, 2)
f

Now we can `lambdify` this $f$ to construct our $b$.

In [ ]:
f_eval = lambdify(x, f)

In [ ]:
def solve_Av_b(n, f):
    
    A = make_A(n)
    
    x = np.linspace(0, 1, n+2)
    h = x[1]-x[0]
    b = h**2*f(x[1:-1])
    
    v = np.zeros(n+2)
    v[1:-1] = np.linalg.solve(A, b)
    
    return v, x, h

In [ ]:
def plot_v_u(n, f, u=None, fignum=2):
    
    plt.figure(fignum)
    plt.clf()
    
    v, x, h = solve_Av_b(n, f)
    
    if u is not None:
        plt.plot(x, u(x), c='b', label='$u$')
    
    plt.plot(x, v, ls=':', c='r', label='$v$')
    
    E_h = np.linalg.norm(np.abs(u(x) - v))  # Two wasted computations here, oh well.
    
    plt.title(r'$h\approx${:1.3f} for $n=${:d} gives $E_h\approx$ {:1.2e}'.format(h, n, E_h) )
    
    plt.legend()
    plt.show()
    return

In [ ]:
%matplotlib widget

%reset -f out 

# Check out what happens when $h<1e-4$

interact_manual(plot_v_u, 
            n=widgets.IntSlider(value=5, min=3, max=100, step=1),
            f = fixed(f_eval),
            u = fixed(lambdify(x, u)),
            fignum = fixed(2))

---
#### Student Activity
---

What if a function that we want to use as a manufactured solution does not satisfy the BCs? 

We can adjust any $w\in\mathcal{C}^2((0,1))$ that has finite $w(0)$ and $w(1)$ to satisfy the BCs by simply defining $u(x)=w(x)-\ell(x)$ where $\ell(x)$ is the linear function whose graph corresponds to the line connecting $w(0)$ to $w(1)$. 

Conceptually, this defines a $u$ that is essentially $w$ except the $x$-axis can be thought of as being "re-defined" to be the line given by the graph of $\ell(x)$. 

1. For the following $w$, define the appropriate $u$ as a manufactured solution and compare $u$ to $v$ as was done above. (We will do a few in class.)

2. Make loglog-plots of $E_h$ vs. $h$ and compute the rate of convergence. (Students are left to do this on their own.)

The $w$ are:

(a) $w(x)=x^3$.

(b) $w(x)=e^x\sin(3\pi x/2)$.

(c) $w(x)=\tan(x)$.

**A suggestion (and possibly a homework problem)**

<mark>Students are encouraged to build upon the `solve_2pt_BVP_Dirichlet` class from [Section 2.1](Chp2Sec1.ipynb) to include numerical estimates of $u$ as well. This should include new data attributes (e.g., $n$, $h$, $v$ and $A$) and method attributes such as `make_A` and `solve_Av_b`.</mark>

---
### <a id='Section2.2.3'>Section 2.2.4: Some properties of $A$</a>
---

Here, we simply list some important and useful properties of $A$ for the particular problem considered in this section. Students may want to refer to [Section 1.5](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp1/Chp1Sec5.ipynb) for the linear algebra details.

- $A$ is symmetric positive definite so is nonsingular and has all positive real eigenvalues.

  - This implies that $v$ always exist and is unique for any $b\in\mathbb{R}^n$. 
  
  - This also implies that $v$ exists even when $b$ is not associated with any meaningful/practical $f$ for which a solution $u$ can even be considered.
  
Wait, what does it mean that $v$ exists even if $f$ is complete nonsense?

Imagine we define $f$ as follows:

- For any given $x$, take the first 3 decimals to set the random seed and generate $f(x)$ as the first normally distributed random number generated fromtaht seed. This $f$ is nonsense and "arbitrarily rough" (i.e., not smooth or even continuous in the slightest). Yet, $f(x)$ is well-defined and we can produce a $v$ for any given $n$ even though $u$ fails to exist in any meaningful way.

- For any given $x$, define $f(x)=0$ if $x$ is rational and $f(x)=1$ if $x$ is irrational (or vice versa). Then $f$ is *mostly* $1$, but is discontinuous everywhere. Yet, on a computer, $f(x)=0$ for all $x$ we can "compute" so $b$ is always the zero vector and $v$ will also always be the zero vector despite $u$ failing to exist as a function in $\mathcal{C}^2((0,1))$. However, we can define a $u\in\mathcal{C}^2((0,1))$ such that $-u''=f$ almost everywhere. Consider $u(x)=\frac{1}{2}x(1-x)$, which is very clearly not zero anywhere in $(0,1)$ despite $v$ always being that way.

What are the lessons here?

- Just because we solve a discrete problem and get a nice answer does not mean that the continuous problem is reasonable, or at least it may mean that what it means to solve the continuous problem has to be considered from a more sophisticated perspective. (Consider this an advertisement for a more modern PDEs course.)

- We should really consider solutions to the continuous problems from a theoretical perspective *first* and then consider numerical approximations *second*. By this, I mean it is actually practically important to have theoretical justifications for the solution to the *exact* problem existing and being unique before we go about numerically approximating a solution that either does not exist or does not exist in the sense that we designed the numerical method to consider. 

  - Using finite element methods and modern PDE theory we can handle both of the scenarios "imagined" above. 
  
    In the first case, the modern PDE theory would basically tell us, "This is a stupid problem with no solution." This is good to know. Not all problems have solutions. This probably means that if this was a model for some application, then you either have something fundamentally wrong with the model or you have some fundamental misunderstanding of the application. Whatever it is, the application and model clearly require further study.
    
    In the second case, modern PDE theory says the solution exists and is unique in the sense I described it above, but it won't tell us what it is. This is where finite element methods come in that use a Lebesgue integral to "smooth out" the roughness of the function $f$ by actually identifying it as equivalent to $f(x)\equiv 1$ (there is a delicate interplay between the finite element method and the "weak" or "variational" form of the PDE that we would study in modern PDE theory) and the numerical implementation would reveal a new type of $v$ that actually approximates the $u$ quite well. 
    
- The meta-lesson here is that your journey is not over with this class. There is still much to learn. 

---
#### Navigation:

- [Previous](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp2/Chp2Sec1.ipynb)

- [Next](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp2/Chp2Sec3.ipynb)
---